In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.compose import make_column_selector as selector
import joblib
from scipy.stats import mode
import h3
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
import shap
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

In [69]:
data = pd.read_csv('global_fishing_data_1.csv').drop('Unnamed: 0',axis=1)

In [70]:

display(data.head(3))
display(data.isna().sum())
display(data.info(verbose=True))

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source,...,hexagon_coordi_7,hexagon_coordi_7_3,course_hex_level_3_avg,is_fishing_hex_level_3_avg,ship_type_hex_level_3_avg,speed_hex_level_3_avg,course_hex_level_7_avg,is_fishing_hex_level_7_avg,ship_type_hex_level_7_avg,speed_hex_level_7_avg
0,1.252340e+12,2015-01-01 05:08:23,0.0,0.0,0.0,128.0,52.458717,4.581316,0.0,gfw,...,87196820affffff,831968fffffffff,120.599998,0.001737,trawlers,0.358636,120.599998,0.0,trawlers,0.007767
1,1.252340e+12,2015-01-01 05:20:34,0.0,0.0,0.0,128.0,52.458733,4.581316,0.0,gfw,...,87196820affffff,831968fffffffff,120.599998,0.001737,trawlers,0.358636,120.599998,0.0,trawlers,0.007767
2,1.252340e+12,2015-01-01 05:32:53,0.0,0.0,0.0,128.0,52.458698,4.581267,0.0,gfw,...,87196820affffff,831968fffffffff,120.599998,0.001737,trawlers,0.358636,120.599998,0.0,trawlers,0.007767


mmsi                          0
timestamp                     0
distance_from_shore           0
distance_from_port            0
speed                         2
course                        2
lat                           0
lon                           0
is_fishing                    0
source                        0
ship_type                     0
hour                          0
day                           0
month                         0
year                          0
hexagon_coordi_7              0
hexagon_coordi_7_3            0
course_hex_level_3_avg        0
is_fishing_hex_level_3_avg    0
ship_type_hex_level_3_avg     0
speed_hex_level_3_avg         0
course_hex_level_7_avg        1
is_fishing_hex_level_7_avg    0
ship_type_hex_level_7_avg     0
speed_hex_level_7_avg         1
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553855 entries, 0 to 553854
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   mmsi                        553855 non-null  float64
 1   timestamp                   553855 non-null  object 
 2   distance_from_shore         553855 non-null  float64
 3   distance_from_port          553855 non-null  float64
 4   speed                       553853 non-null  float64
 5   course                      553853 non-null  float64
 6   lat                         553855 non-null  float64
 7   lon                         553855 non-null  float64
 8   is_fishing                  553855 non-null  float64
 9   source                      553855 non-null  object 
 10  ship_type                   553855 non-null  object 
 11  hour                        553855 non-null  int64  
 12  day                         553855 non-null  int64  
 13  month         

None

In [71]:
data.set_index('mmsi',inplace=True)

In [72]:
data.columns

Index(['timestamp', 'distance_from_shore', 'distance_from_port', 'speed',
       'course', 'lat', 'lon', 'is_fishing', 'source', 'ship_type', 'hour',
       'day', 'month', 'year', 'hexagon_coordi_7', 'hexagon_coordi_7_3',
       'course_hex_level_3_avg', 'is_fishing_hex_level_3_avg',
       'ship_type_hex_level_3_avg', 'speed_hex_level_3_avg',
       'course_hex_level_7_avg', 'is_fishing_hex_level_7_avg',
       'ship_type_hex_level_7_avg', 'speed_hex_level_7_avg'],
      dtype='object')

In [73]:
data.drop(['timestamp','hexagon_coordi_7','hexagon_coordi_7_3','is_fishing_hex_level_3_avg','is_fishing_hex_level_7_avg','lat','lon'],axis=1,inplace=True)

In [74]:
data = data.query('is_fishing == 0 or is_fishing == 1')

In [75]:
data.isna().sum()

distance_from_shore          0
distance_from_port           0
speed                        2
course                       2
is_fishing                   0
source                       0
ship_type                    0
hour                         0
day                          0
month                        0
year                         0
course_hex_level_3_avg       0
ship_type_hex_level_3_avg    0
speed_hex_level_3_avg        0
course_hex_level_7_avg       1
ship_type_hex_level_7_avg    0
speed_hex_level_7_avg        1
dtype: int64

In [76]:
data.dropna(inplace=True)

In [77]:
target_col = 'is_fishing'

In [34]:
data.groupby('ship_type')[['is_fishing']].mean()

,is_fishing
ship_type,
drifting_longlines,0.634541
fixed_gear,0.263626
pole_and_line,0.207782
purse_seines,0.121902
trawlers,0.354028
trollers,0.384595
unknown,0.396557


In [78]:
categorical_columns = ['source','ship_type_hex_level_3_avg','ship_type_hex_level_7_avg','ship_type'] #ship_type ommitted for experiment

In [79]:

data = pd.get_dummies(data, columns=categorical_columns)

In [80]:
data['speed'].fillna(data['speed'].median(), inplace=True)
data['course'].fillna(data['course'].median(), inplace=True)

In [20]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 543475 entries, 1252339803566.0 to 43935946737362.0
Data columns (total 40 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   distance_from_shore                           543475 non-null  float64
 1   distance_from_port                            543475 non-null  float64
 2   speed                                         543475 non-null  float64
 3   course                                        543475 non-null  float64
 4   is_fishing                                    543475 non-null  float64
 5   hour                                          543475 non-null  int64  
 6   day                                           543475 non-null  int64  
 7   month                                         543475 non-null  int64  
 8   year                                          543475 non-null  int64  
 9   course_hex_level_3_avg       

In [24]:
meta_params = {'n_estimators': 500, 'max_depth': 5, 'random_state': 2024}
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(**meta_params),
    'Naive Bayes': GaussianNB(),
    'Ridge Classifier': RidgeClassifier()
}


In [81]:
X = data.drop(target_col, axis=1)
y = data[target_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
X_train.head(2).T

mmsi,1.946720e+14,2.690503e+14
distance_from_shore,434215.09375,281959.375
distance_from_port,445101.40625,1217990.25
speed,0.7,4.1
course,306.0,114.0
hour,2,8
day,3,20
month,12,7
year,2014,2015
course_hex_level_3_avg,86.599998,122.599998
speed_hex_level_3_avg,4.956129,3.974208


In [28]:
pd.DataFrame(X_train.isna().sum())

,0
distance_from_shore,0
distance_from_port,0
speed,0
course,0
hour,0
day,0
month,0
year,0
course_hex_level_3_avg,0
speed_hex_level_3_avg,0


In [32]:

# Dictionary to store the results
results = {}

# Training and evaluating models
for name, model in models.items():
    print(f'----------{name}---------')
    # Create a pipeline for each model
    pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('classifier', model)])
    # Fit the pipeline
    pipeline.fit(X_train, y_train)
    # Predicting and evaluating
    y_pred = pipeline.predict(X_test)
    y_proba = pipeline.predict_proba(X_test)[:, 1] if hasattr(pipeline, "predict_proba") else pipeline.decision_function(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    roc_auc = roc_auc_score(y_test, y_proba)
    report['roc_auc'] = roc_auc
    results[name] = report
    # Save the model configuration
    joblib.dump(pipeline, f'{name}_model.pkl')



----------Logistic Regression---------
----------Decision Tree---------
----------Random Forest---------
----------Naive Bayes---------
----------Ridge Classifier---------


In [36]:
rows = []
for model_name, metrics in results.items():
    for metric, values in metrics.items():
        if isinstance(values, dict):
            for sub_metric, value in values.items():
                rows.append([model_name, f"{metric}_{sub_metric}", value])
        else:
            rows.append([model_name, metric, values])

# Create DataFrame
df_results = pd.DataFrame(rows, columns=["Model", "Metric", "Value"])
df_results = df_results.pivot(index="Model", columns="Metric", values="Value")

# Display the DataFrame
df_results

Metric,0.0_f1-score,0.0_precision,0.0_recall,0.0_support,1.0_f1-score,1.0_precision,1.0_recall,1.0_support,accuracy,macro avg_f1-score,macro avg_precision,macro avg_recall,macro avg_support,roc_auc,weighted avg_f1-score,weighted avg_precision,weighted avg_recall,weighted avg_support
Model,,,,,,,,,,,,,,,,,,
Decision Tree,0.974637,0.975867,0.973410,59196.0,0.969752,0.968297,0.971212,49499.0,0.972409,0.972194,0.972082,0.972311,108695.0,0.972309,0.972412,0.972419,0.972409,108695.0
Logistic Regression,0.768945,0.748103,0.790982,59196.0,0.705674,0.731640,0.681489,49499.0,0.741120,0.737310,0.739871,0.736236,108695.0,0.801496,0.740132,0.740606,0.741120,108695.0
Naive Bayes,0.377856,0.808150,0.246571,59196.0,0.657004,0.507911,0.929999,49499.0,0.557799,0.517430,0.658031,0.588285,108695.0,0.799269,0.504978,0.671423,0.557799,108695.0
Random Forest,0.874956,0.945929,0.813889,59196.0,0.871612,0.809269,0.944363,49499.0,0.873306,0.873284,0.877599,0.879126,108695.0,0.938832,0.873433,0.883695,0.873306,108695.0
Ridge Classifier,0.767233,0.745754,0.789986,59196.0,0.702839,0.729669,0.677913,49499.0,0.738948,0.735036,0.737711,0.733949,108695.0,0.798577,0.737908,0.738429,0.738948,108695.0


In [37]:
df_results.to_excel('Benchmark_of_Algorithms_for_All_data.xlsx')

In [38]:
del data

In [39]:
import gc
gc.collect()

0

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
import joblib
results = {}
meta_params = {'max_depth': 2, 'random_state': 2024}
X = data.drop(target_col, axis=1)
y = data[target_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Decision Tree
dt_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('classifier', DecisionTreeClassifier(max_depth=2))])
dt_pipeline.fit(X_train, y_train)
y_pred_dt = dt_pipeline.predict(X_test)
train_report_dt = classification_report(y_train, dt_pipeline.predict(X_train), output_dict=True)
test_report_dt = classification_report(y_test, y_pred_dt, output_dict=True)
roc_auc_dt = roc_auc_score(y_test, dt_pipeline.predict_proba(X_test)[:, 1])
test_report_dt['roc_auc'] = roc_auc_dt
results['Decision Tree'] = {'train': train_report_dt, 'test': test_report_dt}
joblib.dump(dt_pipeline, 'Decision_Tree_model.pkl')

# Random Forest
rf_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('classifier', RandomForestClassifier(max_depth=5))])
rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)
y_proba_rf = rf_pipeline.predict_proba(X_test)[:, 1]
train_report_rf = classification_report(y_train, rf_pipeline.predict(X_train), output_dict=True)
test_report_rf = classification_report(y_test, y_pred_rf, output_dict=True)
roc_auc_rf = roc_auc_score(y_test, y_proba_rf)
test_report_rf['roc_auc'] = roc_auc_rf
results['Random Forest'] = {'train': train_report_rf, 'test': test_report_rf}
joblib.dump(rf_pipeline, 'Random_Forest_model.pkl')



# Display the results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    print("Train Results:")
    print(metrics['train'])
    print("Test Results:")
    print(metrics['test'])
    print("\n")


Model: Decision Tree
Train Results:
{'0.0': {'precision': 0.9279258460432394, 'recall': 0.7631281094002078, 'f1-score': 0.837496987337548, 'support': 236782.0}, '1.0': {'precision': 0.7663528431576755, 'recall': 0.9291154456105617, 'f1-score': 0.8399216524216524, 'support': 197998.0}, 'accuracy': 0.8387184323105938, 'macro avg': {'precision': 0.8471393446004574, 'recall': 0.8461217775053848, 'f1-score': 0.8387093198796002, 'support': 434780.0}, 'weighted avg': {'precision': 0.8543458022847042, 'recall': 0.8387184323105938, 'f1-score': 0.8386011752885174, 'support': 434780.0}}
Test Results:
{'0.0': {'precision': 0.9274282900406254, 'recall': 0.7635819987837016, 'f1-score': 0.8375674023014064, 'support': 59196.0}, '1.0': {'precision': 0.7665827176142902, 'recall': 0.928544010990121, 'f1-score': 0.8398260488232715, 'support': 49499.0}, 'accuracy': 0.8387046322277933, 'macro avg': {'precision': 0.8470055038274578, 'recall': 0.8460630048869113, 'f1-score': 0.838696725562339, 'support': 1086

In [86]:
def results_to_df(results):
    rows = []
    for model, result in results.items():
        for dataset, metrics in result.items():
            for metric, values in metrics.items():
                if isinstance(values, dict):
                    for label, score in values.items():
                        rows.append({
                            'Model': model,
                            'Dataset': dataset,
                            'Metric': metric,
                            'Label': label,
                            'Score': score
                        })
                else:
                    rows.append({
                        'Model': model,
                        'Dataset': dataset,
                        'Metric': metric,
                        'Label': 'overall',
                        'Score': values
                    })
    return pd.DataFrame(rows)

results_to_df(results).to_excel('Benchmark_of_Algorithms_for_All_data_v2.xlsx')

In [ ]:
# Logistic Regression
lr_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('classifier', LogisticRegression(max_iter=1000))])
lr_pipeline.fit(X_train, y_train)
y_pred_lr = lr_pipeline.predict(X_test)
y_proba_lr = lr_pipeline.predict_proba(X_test)[:, 1]
train_report_lr = classification_report(y_train, lr_pipeline.predict(X_train), output_dict=True)
test_report_lr = classification_report(y_test, y_pred_lr, output_dict=True)
roc_auc_lr = roc_auc_score(y_test, y_proba_lr)
test_report_lr['roc_auc'] = roc_auc_lr
results['Logistic Regression'] = {'train': train_report_lr, 'test': test_report_lr}
joblib.dump(lr_pipeline, 'Logistic_Regression_model.pkl')
# Naive Bayes
nb_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('classifier', GaussianNB())])
nb_pipeline.fit(X_train, y_train)
y_pred_nb = nb_pipeline.predict(X_test)
y_proba_nb = nb_pipeline.predict_proba(X_test)[:, 1]
train_report_nb = classification_report(y_train, nb_pipeline.predict(X_train), output_dict=True)
test_report_nb = classification_report(y_test, y_pred_nb, output_dict=True)
roc_auc_nb = roc_auc_score(y_test, y_proba_nb)
test_report_nb['roc_auc'] = roc_auc_nb
results['Naive Bayes'] = {'train': train_report_nb, 'test': test_report_nb}
joblib.dump(nb_pipeline, 'Naive_Bayes_model.pkl')

# Ridge Classifier
rc_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('classifier', RidgeClassifier())])
rc_pipeline.fit(X_train, y_train)
y_pred_rc = rc_pipeline.predict(X_test)
train_report_rc = classification_report(y_train, rc_pipeline.predict(X_train), output_dict=True)
test_report_rc = classification_report(y_test, y_pred_rc, output_dict=True)
roc_auc_rc = roc_auc_score(y_test, rc_pipeline.decision_function(X_test))
test_report_rc['roc_auc'] = roc_auc_rc
results['Ridge Classifier'] = {'train': train_report_rc, 'test': test_report_rc}
joblib.dump(rc_pipeline, 'Ridge_Classifier_model.pkl')


In [83]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
#    'classifier__max_features': ['auto', 'sqrt', 'log2']
}

# Create a pipeline with StandardScaler and RandomForestClassifier
rf_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('classifier', RandomForestClassifier(random_state=2024))])

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=rf_pipeline, param_grid=param_grid, cv=3, n_jobs=-1, scoring='f1_weighted')
grid_search.fit(X_train, y_train)

# Print the best parameters
print(f"Best parameters: {grid_search.best_params_}")




KeyboardInterrupt: 

In [ ]:
# Evaluate the best model
best_rf_pipeline = grid_search.best_estimator_
y_pred_rf = best_rf_pipeline.predict(X_test)
y_proba_rf = best_rf_pipeline.predict_proba(X_test)[:, 1]
train_report_rf = classification_report(y_train, best_rf_pipeline.predict(X_train), output_dict=True)
test_report_rf = classification_report(y_test, y_pred_rf, output_dict=True)
roc_auc_rf = roc_auc_score(y_test, y_proba_rf)
test_report_rf['roc_auc'] = roc_auc_rf
results['Random Forest (Tuned)'] = {'train': train_report_rf, 'test': test_report_rf}

# Display the results
print("Random Forest (Tuned) Results")
print("Train Results:")
print(train_report_rf)
print("Test Results:")
print(test_report_rf)

In [45]:
def results_to_df(results):
    rows = []
    for model, result in results.items():
        for dataset, metrics in result.items():
            for metric, values in metrics.items():
                if isinstance(values, dict):
                    for label, score in values.items():
                        rows.append({
                            'Model': model,
                            'Dataset': dataset,
                            'Metric': metric,
                            'Label': label,
                            'Score': score
                        })
                else:
                    rows.append({
                        'Model': model,
                        'Dataset': dataset,
                        'Metric': metric,
                        'Label': 'overall',
                        'Score': values
                    })
    return pd.DataFrame(rows)

results_to_df(results).to_excel('Benchmark_of_Algorithms_for_All_data_v1.xlsx')

In [25]:
data.sample(1000).to_csv('Sample_Shipping_Data.csv')

In [22]:
data.columns

Index(['distance_from_shore', 'distance_from_port', 'speed', 'course',
       'is_fishing', 'hour', 'day', 'month', 'year', 'course_hex_level_3_avg',
       'speed_hex_level_3_avg', 'course_hex_level_7_avg',
       'speed_hex_level_7_avg', 'source_crowd_sourced',
       'source_dalhousie_longliner', 'source_dalhousie_ps',
       'source_dalhousie_trawl', 'source_false_positives', 'source_gfw',
       'ship_type_hex_level_3_avg_drifting_longlines',
       'ship_type_hex_level_3_avg_fixed_gear',
       'ship_type_hex_level_3_avg_pole_and_line',
       'ship_type_hex_level_3_avg_purse_seines',
       'ship_type_hex_level_3_avg_trawlers',
       'ship_type_hex_level_3_avg_trollers',
       'ship_type_hex_level_3_avg_unknown',
       'ship_type_hex_level_7_avg_drifting_longlines',
       'ship_type_hex_level_7_avg_fixed_gear',
       'ship_type_hex_level_7_avg_pole_and_line',
       'ship_type_hex_level_7_avg_purse_seines',
       'ship_type_hex_level_7_avg_trawlers',
       'ship_type_hex_

### GWR Application

In [37]:
data = pd.read_csv('global_fishing_data_1.csv').drop('Unnamed: 0',axis=1)

In [38]:
data_sampled = data.sample(1000)

In [39]:
data_sampled.drop(['timestamp','hexagon_coordi_7','hexagon_coordi_7_3','is_fishing_hex_level_3_avg','is_fishing_hex_level_7_avg'],axis=1,inplace=True)

In [40]:
target_col = 'is_fishing'

In [41]:
data_sampled.dropna(inplace=True)

In [59]:


from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap
import numpy as np
import statsmodels.api as sm

# Define the coordinates and dependent/independent variables
coords = np.array(list(zip(data_sampled['lon'], data_sampled['lat'])))
y = data_sampled[target_col].values.reshape((-1, 1))  # Replace with actual dependent variable column


In [60]:
#data.drop(['timestamp','hexagon_coordi_7','hexagon_coordi_7_3','is_fishing_hex_level_3_avg','is_fishing_hex_level_7_avg'],axis=1,inplace=True)
X = data_sampled[['distance_from_shore', 'distance_from_port', 'speed', 'course','course_hex_level_3_avg',
       'speed_hex_level_3_avg', 'course_hex_level_7_avg','speed_hex_level_7_avg']].values  # Replace with actual independent variables


In [61]:
X = sm.add_constant(X)

In [62]:
X[0],X[1],X[2],X[3]

(array([1.00000000e+00, 3.57762109e+04, 1.00242234e+05, 3.70000005e+00,
        3.24000015e+01, 0.00000000e+00, 2.94024817e+00, 3.24000015e+01,
        6.95000000e+00]),
 array([1.00000000e+00, 2.64757539e+04, 3.71608945e+04, 1.06000004e+01,
        1.90000000e+01, 3.08000000e+02, 2.09845560e+00, 1.90000000e+01,
        7.65000010e+00]),
 array([1.00000000e+00, 1.14233789e+05, 2.14099859e+05, 6.19999981e+00,
        3.13299988e+02, 3.17200012e+02, 5.54130436e+00, 3.13299988e+02,
        6.19999997e+00]),
 array([1.00000000e+00, 1.01594750e+05, 1.12695953e+05, 1.89999998e+00,
        1.10599998e+02, 7.90999985e+01, 3.80060096e+00, 1.10599998e+02,
        3.03333334e+00]))

In [63]:
selector = Sel_BW(coords, y, X)
bandwidth = selector.search()

# Fit the GWR model
gwr_model = GWR(coords, y, X, bw=bandwidth)
gwr_results = gwr_model.fit()

In [64]:

print(gwr_results.summary())


Model type                                                         Gaussian
Number of observations:                                                1000
Number of covariates:                                                     9

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                            209.261
Log-likelihood:                                                    -636.853
AIC:                                                               1291.705
AICc:                                                              1293.928
BIC:                                                              -6636.324
R2:                                                                   0.155
Adj. R2:                                                              0.148

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [65]:
predictions = gwr_model.predict(coords, X)

In [ ]:
predictions

ValueError: operands could not be broadcast together with shapes (1000,10) (1000,9) 